# Donkey Colab with GPU

Donkey Colab with GPU.

# TensorFlowをVersion Down

Colab環境では、TensorFlowが1.13.0になっているので、1.12.0にDown gradeします。

In [1]:
!pip install tensorflow==1.12.0
!pip install q keras==2.2.4

    100% |████████████████████████████████| 83.1MB 440kB/s 
  Found existing installation: tensorflow 1.13.0rc2
    Uninstalling tensorflow-1.13.0rc2:
      Successfully uninstalled tensorflow-1.13.0rc2


# DonkeyレポジトリをGit clone

In [2]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 6742 (delta 12), reused 24 (delta 9), pack-reused 6702
Receiving objects: 100% (6742/6742), 42.65 MiB | 21.75 MiB/s, done.
Resolving deltas: 100% (3875/3875), done.


# インストール

In [3]:
!pip3 install -e donkey

Obtaining file:///content/donkey
    100% |████████████████████████████████| 51kB 4.6MB/s 
    100% |████████████████████████████████| 225kB 15.4MB/s 
    100% |████████████████████████████████| 81kB 26.6MB/s 
    100% |████████████████████████████████| 194kB 31.0MB/s 
  Running setup.py develop for donkeycar


# Projectの作成

In [4]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


# datas.zipのアップロード
下記コードを実行すると、ファイルのアップロード フォームが出現します。datas.zipをアップリードします。

In [5]:
from google.colab import files
uploaded = files.upload()

Saving datas.zip to datas.zip


# datas.zipの移動と解答
datas.zipは、/content以下にアップされます。/content/mycar/dataに移動し、解凍します。

In [0]:
!mv /content/datas.zip /content/mycar/data/

In [7]:
%cd /content/mycar/data/

!unzip datas.zip

/content/mycar/data
Archive:  datas.zip
   creating: tub_01_19-02-26/
  inflating: tub_01_19-02-26/meta.json  
   creating: tub_02_19-02-26/
  inflating: tub_02_19-02-26/277_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/record_204.json  
  inflating: tub_02_19-02-26/record_284.json  
  inflating: tub_02_19-02-26/record_233.json  
  inflating: tub_02_19-02-26/21_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/record_292.json  
  inflating: tub_02_19-02-26/record_254.json  
  inflating: tub_02_19-02-26/record_242.json  
  inflating: tub_02_19-02-26/233_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/record_73.json  
  inflating: tub_02_19-02-26/record_308.json  
  inflating: tub_02_19-02-26/record_230.json  
  inflating: tub_02_19-02-26/125_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/15_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/173_cam-image_array_.jpg  
  inflating: tub_02_19-02-26/record_253.json  
  inflating: tub_02_19-02-26/record_298.json  
  inflati

# 学習

学習する前に、ここまでインストールしたライブラリを有効にするために
**ランタイム-ランタイムの再起動**
を実行します。

In [4]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

tub_names None
train: 289, validation: 73
steps_per_epoch 2
Epoch 1/100
1/2 [==============>...............] - ETA: 3s - loss: 239.0763 - angle_out_loss: 215.7260 - throttle_out_loss: 262.4266
Epoch 00001: val_loss improved from inf to 64.54079, saving model to /content/mycar/models/mypilot
2/2 [==============================] - 8s 4s/step - loss: 2527.5272 - angle_out_loss: 1554.2816 - throttle_out_loss: 3500.7729 - val_loss: 64.5408 - val_angle_out_loss: 6.4701 - val_throttle_out_loss: 122.6115
Epoch 2/100
1/2 [==============>...............] - ETA: 1s - loss: 72.7936 - angle_out_loss: 14.2013 - throttle_out_loss: 131.3860
Epoch 00002: val_loss improved from 64.54079 to 21.77225, saving model to /content/mycar/models/mypilot
2/2 [==============================] - 5s 3s/step - loss: 51.2324 - angle_out_loss: 35.1728 - throttle_out_loss: 67.2919 - val_loss: 21.7723 - val_angle_out_loss: 41.4011 - val_throttle_out_loss: 2.1434
Epoch 3/100
1/2 [==============>...............] - ETA: 1s -